## **Pipline**

1. feature extractor
2. offence model
3. foul type model
4. convert predictions

In [2]:
import torch
from transformers import pipeline
import numpy as np
import keras

In [13]:
offence_class = {
    0:"No offence", 
    1:"Offence + No card", 
    2:"Offence + Yellow card", 
    3:"Offence + Red card"
}

In [14]:
action_class = {
    0:"Tackling", 
    1:"Standing tackling", 
    2:"High leg", 
    3:"Holding", 
    4:"Pushing",
    5:"Elbowing", 
    6:"Challenge", 
    7:"Dive", 
    8:"Dont know"
}

#### **Feature Extractor**
input: video

output: feature vector

In [3]:
#Download model
pipe_soccer = pipeline("video-classification", model="anirudhmu/videomae-base-finetuned-soccer-action-recognitionx4")

pipe_soccer.model.classifier = torch.nn.Identity() #removes the final layer
pipe_soccer.postprocess = lambda x, *a, **k: x.logits #get logits

In [7]:
offence_model= keras.models.load_model('./multi_offence_model.keras')
foul_model= keras.models.load_model('./multi_foul_model2.keras')

In [44]:
def extract(video_path):
    action_features=[]
    clip_features= pipe_soccer.predict(video_path)
    action_features.append(clip_features)

    features= np.asarray(action_features)
    #prepare data to model
    features= np.reshape(features, (1, 768))

    #predict
    offence_pred= offence_model.predict(features)
    foul_pred= foul_model.predict(features)

    #offence
    offence_max_pred= (offence_pred[0]).max()
    offence_max_index= np.argmax(offence_pred[0])

    #foul
    foul_max_pred= (foul_pred[0]).max()
    foul_max_index= np.argmax(foul_pred[0])

    predictions={
        'offence': [offence_max_pred, offence_class[offence_max_index]],
        'foul': [foul_max_pred, action_class[foul_max_index]]
    }

    return predictions
    

In [42]:
#prepare data to model
features= np.reshape(features, (1, 768))

#### **Models Prediction**

1. offence model

2. foul model

In [43]:
#predict
offence_pred= offence_model.predict(features)
foul_pred= foul_model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


In [36]:
#take max 2 classes
offence_max_pred= (offence_pred[0]).max()
offence_max_index= np.argmax(offence_pred[0])

In [37]:
#take max 2 classes
foul_max_pred= (foul_pred[0]).max()
foul_max_index= np.argmax(foul_pred[0])

In [38]:
predictions={
    'offence': [offence_max_pred, offence_class[offence_max_index]],
    'foul': [foul_max_pred, action_class[foul_max_index]]
}

In [39]:
predictions

{'offence': [0.2736536, 'Offence + No card'], 'foul': [0.22497404, 'Elbowing']}